<a href="https://colab.research.google.com/github/tgngenuka/Artificial-Intelligence-Bootcamp-TechCrush/blob/main/faw_caps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Imports and installs

In [4]:
# --- 1. SETUP: Install necessary libraries ---
!pip install tf2onnx

# --- 2. IMPORTS ---
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import tf2onnx
import onnx

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


2. Google Drive Mount

In [5]:
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/FAW_dataset"

IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 32
RANDOM_SEED = 42
EPOCHS = 10

# Set a consistent random seed for reproducibility
tf.random.set_seed(RANDOM_SEED)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3. Data Loading amd Preprocessing

In [9]:
# Use ImageDataGenerator for efficient loading and data augmentation
# Data Augmentation is strongly encouraged as per the project description
data_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values to [0, 1]
    validation_split=0.3, # 30% for validation/test sets
    # Simple Augmentation techniques:
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

4. Training Data Generator

In [10]:
print("\n--- Loading Training Data ---")
train_generator = data_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary', # For binary classification (FAW vs No-FAW)
    subset='training', # Specify training subset
    seed=RANDOM_SEED
)

# 5.2 Validation/Test Data Generator (Split into two separate steps if needed,
# but for simplicity, we'll use one generator for validation/test for now)
print("\n--- Loading Validation/Test Data ---")
val_test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

val_generator = val_test_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation', # Use first half (e.g., 50% of 30% = 15% overall) for validation
    seed=RANDOM_SEED
)

# Separate Test Set (A common practice is to reserve 10-20% for testing)
# Due to the constraints of `flow_from_directory`'s `validation_split`,
# we'll use a portion of the validation set for testing as an approximation.
# In a full project, you should manually split the data folder into train/val/test.


--- Loading Training Data ---
Found 2091 images belonging to 2 classes.

--- Loading Validation/Test Data ---
Found 1493 images belonging to 2 classes.


5. Model Architecture (Simple Custom CNN)

In [11]:
model = Sequential([
    # First Block
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D((2, 2)),

    # Second Block
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Third Block
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Classification Head
    Flatten(),
    Dropout(0.5), # Regularization to prevent overfitting
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid') # Sigmoid for binary classification
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6. Model Compilation and Training

In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy', # Appropriate loss for binary classification
              metrics=['accuracy'])

model.summary()

print("\n--- Starting Model Training ---")

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

 Trainable params: 12,939,329 (49.36 MB)

 Non-trainable params: 0 (0.00 B)


--- Starting Model Training ---
Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.7924 - loss: 0.4414 - val_accuracy: 0.7840 - val_loss: 0.5517
Epoch 2/10
 1/65 ━━━━━━━━━━━━━━━━━━━━ 1:35 1s/step - accuracy: 0.9062 - loss: 0.2007

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


65/65 ━━━━━━━━━━━━━━━━━━━━ 34s 506ms/step - accuracy: 0.9062 - loss: 0.2007 - val_accuracy: 0.7683 - val_loss: 0.5977
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.8177 - loss: 0.3850 - val_accuracy: 0.8193 - val_loss: 0.5485
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 28s 415ms/step - accuracy: 0.8750 - loss: 0.2648 - val_accuracy: 0.8227 - val_loss: 0.5364
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.8310 - loss: 0.3559 - val_accuracy: 0.7948 - val_loss: 0.4938
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 28s 417ms/step - accuracy: 0.8750 - loss: 0.2384 - val_accuracy: 0.8091 - val_loss: 0.4803
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.8746 - loss: 0.3051 - val_accuracy: 0.8295 - val_loss: 0.5148
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 27s 409ms/step - accuracy: 0.8125 - loss: 0.4642 - val_accuracy: 0.8431 - val_loss: 0.4908
Epoch 9/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8769 - loss: 0.2977 - val_accuracy: 0.8390 - val_

7. Evaluation

In [14]:
val_generator.reset()
Y_pred = model.predict(val_generator)
y_pred_classes = (Y_pred > 0.5).astype(int).flatten()
y_true = val_generator.classes[:len(y_pred_classes)] # Ensure lengths match

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred_classes, target_names=['No-FAW', 'FAW']))

print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

47/47 ━━━━━━━━━━━━━━━━━━━━ 27s 564ms/step

--- Classification Report ---
              precision    recall  f1-score   support

      No-FAW       0.63      0.76      0.69       941
         FAW       0.38      0.25      0.30       552

    accuracy                           0.57      1493
   macro avg       0.50      0.50      0.50      1493
weighted avg       0.54      0.57      0.55      1493


--- Confusion Matrix ---
[[711 230]
 [413 139]]


9. ONNX Export (Deployment Readiness)

In [16]:
ONNX_MODEL_FILE = "faw_detection_model.onnx"

print(f"\n--- Exporting Model to ONNX: {ONNX_MODEL_FILE} ---")

# The input signature must match the model's expected input shape
spec = (tf.TensorSpec((None, IMG_WIDTH, IMG_HEIGHT, 3), tf.float32, name="input"),)

# Define a wrapper function to explicitly set output names
@tf.function(input_signature=spec)
def serving_fn(input_tensor):
    return {'output': model(input_tensor)}


# Convert the Keras model to ONNX format using the wrapper function
onnx_model, _ = tf2onnx.convert.from_function(
    serving_fn,
    input_signature=spec,
    opset=13, # Recommended opset for broad compatibility
    output_path=ONNX_MODEL_FILE
)

# Verify the ONNX model structure
onnx.checker.check_model(onnx_model)

print(f"✅ Model successfully exported and saved as {ONNX_MODEL_FILE}")
print("You can download this file from the Colab file browser for deployment.")


--- Exporting Model to ONNX: faw_detection_model.onnx ---


ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7f9966118540>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


✅ Model successfully exported and saved as faw_detection_model.onnx
You can download this file from the Colab file browser for deployment.
